<a href="https://colab.research.google.com/gist/mohammedkhalilia/a4d83d4e43682d1efcdf299d41beb3da/train_nested_arabic_ner.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Verify that you have the GPU recognized 
!nvidia-smi

In [ ]:
# Install dependencies
!pip install pytorch==1.13.1
!pip install transformers==4.26.1
!pip install torchtext==0.14.0
!pip install torchvision==0.14.1
!pip install seqeval==1.2.2

In [ ]:
# Remove existing package and clone again from Github
!rm -rf /content/ArabicNER
!git clone https://github.com/SinaLab/ArabicNER.git

In [ ]:
# Add the ArabicNER package to the system path
import sys
import argparse
sys.path.append('/content/ArabicNER/')

In [ ]:
# Import train function
from arabiner.bin.train import main as train

In [ ]:
# Setup the model arguments
args_dict = {
    # Model output path to save artifacts and model predictions
    "output_path": "/content/output/",

    # train/test/validation data paths
    "train_path": "/content/ArabicNER/data/train.txt",
    "test_path": "/content/ArabicNER/data/test.txt",
    "val_path": "/content/ArabicNER/data/val.txt",

    # seed for randomization
    "seed": 1,

    "batch_size": 8,

    # Nmber of workers for the dataloader
    "num_workers": 1,

    # GPU/device Ids to train model on
    # For two GPUs use [0, 1]
    # For three GPUs use [0, 1, 2], etc.
    "gpus": [0],

    # Overwrite data in output_path directory specified above
    "overwrite": True,

    # How often to print the logs in terms of number of steps
    "log_interval": 10,

    # Data configuration
    # Here we specify the dataset class and there are two options:
    #  arabiner.data.datasets.DefaultDataset: for flat NER 
    #  arabiner.data.datasets.NestedTagsDataset: for nested NER
    #
    # kwargs: keyword arguments to the dataset class
    # This notebook used the NestedTagsDataset for nested NER
    "data_config": {
        "fn": "arabiner.data.datasets.NestedTagsDataset", 
        "kwargs": {"max_seq_len": 512}
    },

    # Neural net configuration
    # There are two NNs:
    #   arabiner.nn.BertSeqTagger: flat NER tagger
    #   arabiner.nn.BertNestedTagger: nested NER tagger
    #
    # kwargs: keyword arguments to the NN
    # This notebook uses BertNestedTagger for nested NER tagging
    "network_config": {
        "fn": "arabiner.nn.BertNestedTagger", 
        "kwargs": {"dropout": 0.1, "bert_model": "aubmindlab/bert-base-arabertv2"}
    },

    # Model trainer configuration
    #
    #  arabiner.trainers.BertTrainer: for flat NER training
    #  arabiner.trainers.BertNestedTrainer: for nested NER training
    #
    # kwargs: keyword arguments to arabiner.trainers.BertTrainer
    #         additional arguments you can pass includes
    #           - clip: for gradient clpping
    #           - patience: number of epochs for early termination
    # This notebook uses BertNestedTrainer for nested NER training
    "trainer_config": {
        "fn": "arabiner.trainers.BertNestedTrainer", 
        "kwargs": {"max_epochs": 50}
    },

    # Optimizer configuration
    # Our experiments use torch.optim.AdamW, however, you are free to pass
    # any other optmizers such as torch.optim.Adam or torch.optim.SGD
    # lr: learning rate
    # kwargs: keyword arguments to torch.optim.AdamW or whatever optimizer you use
    #
    # Additional optimizers can be found here:
    # https://pytorch.org/docs/stable/optim.html
    "optimizer": {
        "fn": "torch.optim.AdamW", 
        "kwargs": {"lr": 0.0001}
    },

    # Learning rate scheduler configuration
    # You can pass a learning scheduler such as torch.optim.lr_scheduler.StepLR
    # kwargs: keyword arguments to torch.optim.AdamW or whatever scheduler you use
    #
    # Additional schedulers can be found here:
    # https://pytorch.org/docs/stable/optim.html
    "lr_scheduler": {
        "fn": "torch.optim.lr_scheduler.ExponentialLR", 
        "kwargs": {"gamma": 1}
    },

    # Loss function configuration
    # We use cross entropy loss
    # kwargs: keyword arguments to torch.nn.CrossEntropyLoss or whatever loss function you use
    "loss": {
        "fn": "torch.nn.CrossEntropyLoss", 
        "kwargs": {}
    }
}

# Convert args dictionary to argparse namespace
args = argparse.Namespace()
args.__dict__ = args_dict

In [ ]:
# Start training the model
train(args)